# Phase I: Dealing and Playing

In this notebook I write code to deal cards to an arbitrary number of players, let them bet, and find the winner. 

Phase II: I can scale up to make player types and figure out raw probabilities of people having better or worse or the same hands than me. 

Phase III: I can build game theoretic players, RL players, etc. that take into account the results/algorithms from Phase II. 

In [1]:
# IMPORT BOX #
using Random

## Structure: Player Hand

In [2]:
@enum Suit ♣ ♦ ♥ ♠
@enum Rank Two=1 Three Four Five Six Seven Eight Nine Ten Jack Queen King Ace

struct Card
    rank::Rank
    suit::Suit
end

In [3]:
struct Hand
    cards::Array{Card}
end

## Get a Deck of Cards

In [4]:
function get_deck()
    deck = []
    for rnk in (Ace, Two, Three, Four, Five, Six, Seven, Eight, Nine, Ten, Jack, Queen, King)
        for r in (♣, ♦, ♥, ♠)
            push!(deck,Card(rnk,r))
        end
    end
    deck
end

get_deck (generic function with 1 method)

## Deal A Game

In [5]:
deck=get_deck()
shuffle!(deck)

num_players = 5
players = Array{Hand}(undef,num_players)

for i = 1:num_players
    card_array::Array{Card} = [popfirst!(deck), popfirst!(deck)]
    players[i] = Hand(card_array)
end

In [6]:
players

5-element Array{Hand,1}:
 Hand(Card[Card(Two, ♦), Card(Four, ♥)])
 Hand(Card[Card(Three, ♠), Card(Three, ♦)])
 Hand(Card[Card(Jack, ♠), Card(Ten, ♠)])
 Hand(Card[Card(Queen, ♥), Card(Ace, ♠)])
 Hand(Card[Card(Six, ♣), Card(Seven, ♠)])

In [7]:
community_cards = []

Any[]

In [8]:
for i = 1:3
    push!(community_cards, popfirst!(deck))
end

In [9]:
community_cards

3-element Array{Any,1}:
 Card(Two, ♣)
 Card(Ten, ♦)
 Card(Jack, ♣)

In [10]:
push!(community_cards, popfirst!(deck))

4-element Array{Any,1}:
 Card(Two, ♣)
 Card(Ten, ♦)
 Card(Jack, ♣)
 Card(Five, ♣)

In [11]:
push!(community_cards, popfirst!(deck))

5-element Array{Any,1}:
 Card(Two, ♣)
 Card(Ten, ♦)
 Card(Jack, ♣)
 Card(Five, ♣)
 Card(Ace, ♣)

## Dominance Classes of Cards

Let a set of 5 cards be a "full hand". We need a function that compares two full hands. 

Full hands can belong to the following classes: 
 1. Royal flush; 
 2. straight flush; 
      - high_card
 3. four of a kind; 
      - quads_rank
      - off_card
 4. full house; 
      - trips_rank -- # on the triplet.
      - pair_rank -- # on the doubles.
      ------
      - Highest trips win. 
      - Pair is a tiebreaker (e.g., with multiple decks). 
 5. flush; 
      - high card(s) wins.
 6. straight; 
      - high_card.
      - if it's the same high card: tie.
 7. three of a kind; 
      - trips_rank (higher wins)
      - high_card
      - high_card_2
 8. two pair; 
      - high_pair_rank
      - low_pair_rank
      - off_card
 9. one pair; 
      - pair_rank
      - high_card
      - high_card_2
      - high_card_3
 10. high card
      - sort -> compare

There's a nice ordering to these, so $i$ dominates all $j<i$. They're also easy to determine.

If it's a tie, we only really need to resort to the "metadata" for each. 

It's probably more efficient to only get the metadata if necessary (i.e., in the e